# Imports and Constants

## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/') 

## Constants

In [ ]:
#@markdown Training parameters
#@markdown ---
latent_dim = 100 #@param {type:"slider", min:50, max:300, step:50}
#default: 50
NUM_EPOCHS = 30 #@param {type:"slider", min:10, max:400, step:10}  
#default: 128
BATCH_SIZE = 32 #@param {type:"slider", min:32, max:512, step:32}
INIT_LR = 1e-3 #@param {type:"number"}
#@markdown ---

#@markdown Benchmarks
#@markdown --
OUTPUT_PATH = "/content/gdrive/MyDrive/Colab Files/Autoencoder/Fashion MNIST/" #@param {type:"string"}
SAMPLES_NUM = 8 #@param {type:"slider", min:8, max:30, step:1}



## Imports

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

import matplotlib
matplotlib.use("Agg")

from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2

import os

# Download and Prepare The DataSet



In [ ]:
print("[INFO] loading MNIST dataset...")
((trainX, _), (testX, _)) = fashion_mnist.load_data()
trainImages = np.concatenate([trainX, testX])

# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# sample noise from a random normal distribution centered at 0.5
trainNoise = np.random.normal(loc=0.5, scale=0.5, size=trainX.shape)
testNoise = np.random.normal(loc=0.5, scale=0.5, size=testX.shape)
trainXNoisy = np.clip(trainX + trainNoise, 0, 1)
testXNoisy = np.clip(testX + testNoise, 0, 1)



# Building the Model

In [ ]:
def build_autoencoder(width, height, depth, filters=(32, 64), latentDim=16):
	inputShape = (height, width, depth)
	chanDim = -1

	# define the input to the encoder
	inputs = Input(shape=inputShape)
	x = inputs

	# loop over the number of filters
	for f in filters:
		# apply a CONV => RELU => BN operation
		x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
		x = LeakyReLU(alpha=0.2)(x)
		x = BatchNormalization(axis=chanDim)(x)

	# flatten the network and then construct our latent vector
	volumeSize = K.int_shape(x)
	x = Flatten()(x)
	latent = Dense(latentDim)(x)

	# build the encoder model
	encoder = Model(inputs, latent, name="encoder")

	# start building the decoder model which will accept the
	# output of the encoder as its inputs
	latentInputs = Input(shape=(latentDim,))
	x = Dense(np.prod(volumeSize[1:]))(latentInputs)
	x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

	# loop over our number of filters again, but this time in
	# reverse order
	for f in filters[::-1]:
		# apply a CONV_TRANSPOSE => RELU => BN operation
		x = Conv2DTranspose(f, (3, 3), strides=2,
			padding="same")(x)
		x = LeakyReLU(alpha=0.2)(x)
		x = BatchNormalization(axis=chanDim)(x)

	# apply a single CONV_TRANSPOSE layer used to recover the
	# original depth of the image
	x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
	outputs = Activation("sigmoid")(x)

	# build the decoder model
	decoder = Model(latentInputs, outputs, name="decoder")

	# our autoencoder is the encoder + decoder
	autoencoder = Model(inputs, decoder(encoder(inputs)),
		name="autoencoder")

	# return a 3-tuple of the encoder, decoder, and autoencoder
	return (encoder, decoder, autoencoder)

# Training and Testing

In [ ]:
# construct our convolutional autoencoder
print("[INFO] building autoencoder...")
(encoder, decoder, autoencoder) = build_autoencoder(28, 28, 1)
opt = Adam(lr=INIT_LR)
autoencoder.compile(loss="mse", optimizer=opt)

# train the convolutional autoencoder
H = autoencoder.fit(
	trainXNoisy, trainX,
	validation_data=(testXNoisy, testX),
	epochs=NUM_EPOCHS,
	batch_size=BATCH_SIZE)

# construct a plot that plots and saves the training history
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(OUTPUT_PATH + "loss-acc.png")

# use the convolutional autoencoder to make predictions on the
# testing images, then initialize our list of output images
print("[INFO] making predictions...")
decoded = autoencoder.predict(testXNoisy)
outputs = None


images=[]
# loop over our number of output samples
for i in range(0, SAMPLES_NUM):
	# grab the original image and reconstructed image
	original = (testXNoisy[i] * 255).astype("uint8")
	recon = (decoded[i] * 255).astype("uint8")

	# stack the original and reconstructed image side-by-side
	output = np.hstack([original, recon])

	# if the outputs array is empty, initialize it as the current
	# side-by-side image display
	if outputs is None:
		outputs = output

	# otherwise, vertically stack the outputs
	else:
		outputs = np.vstack([outputs, output])


p = OUTPUT_PATH + "output_samples.png" 
# save the outputs image to disk
cv2.imwrite(p, outputs)